In [34]:
import os, glob, shutil, yaml, itertools, math, wandb, sys, json
import numpy as np
import importlib
import re, math
import cv2
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pycocotools.mask as cocomask
import skimage.io as io
import pylab
%matplotlib inline
# import bitsandbytes as bnb
osp = os.path
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

import torch
nn = torch.nn
F = torch.nn.functional
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

import sprelnet
from sprelnet import util
from sprelnet.data import mnist
import mmdet.utils.job_mgmt as omm_util
#import torchvision.datasets

base_ds_folder = '/data/vision/polina/scratch/clintonw/datasets'
mmdet_proj_dir = base_proj_dir = "/data/vision/polina/users/clintonw/code/mmdet"
mmdet_job_dir = base_job_dir = "/data/vision/polina/users/clintonw/code/mmdet/results"
mnist_grid_dir = "/data/vision/polina/scratch/clintonw/datasets/mnist_grid"
coco_dir = "/data/vision/polina/scratch/clintonw/datasets/coco"

## Interactive

### Datasets

#### MNIST

mnist.create_multi_MNIST_dataset()

pixels.create_pixel_dataset();

In [ ]:
ds = util.load_binary_file(pixel_ds_path)
ds["datapoint loader"]= lambda dp: (dp.sum(0, keepdim=True).cuda(), (dp != 0).float().cuda())
util.save_binary_file(data=ds, path=pixel_ds_path)

In [ ]:
importlib.reload(mnist)

In [21]:
digits_per_dim=(5,5)
N_train=90
N_val=10
train_mnist, test_mnist = get_mnist()
base_images = torch.cat([train_mnist[0], test_mnist[0]], 0)
labels = torch.cat([train_mnist[1], test_mnist[1]], 0)
category_names = get_multi_mnist_legend().keys()
n_digits = np.prod(digits_per_dim)

In [23]:
ann_id=0
img_dir=os.path.join(mnist_grid_dir, "train")

In [14]:
mnist.create_mnist_grid_coco_dataset(N_train=4000, N_val=1000)

In [8]:
annFile = os.path.join(mnist_grid_dir, "train/annotation_coco.json")
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [ ]:
imgIds = [2]
img = coco.loadImgs(imgIds)[0]

# load and display image
I = io.imread('%s/%s/%s'%(mnist_grid_dir,"train",img['file_name']), as_gray=True)
# I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I, cmap="gray")
plt.show()

#### COCO

In [35]:
coco_annot_json_dir = "/data/vision/polina/scratch/clintonw/datasets/coco/annotations"
coco_annot_json_path = os.path.join(coco_annot_json_dir, "instances_val2017.json")
# with open(coco_annot_json_path, "r") as f:
#     all_annotations = json.load(f)
coco=cocoGt=COCO(coco_annot_json_path)

In [ ]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

supercats = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(supercats)))

In [43]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','dog','skateboard']);
imgIds = coco.getImgIds(catIds=catIds );
# imgIds = coco.getImgIds(imgIds = [324158])
# img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]

In [ ]:
imgIds

In [ ]:
# load and display instance annotations
plt.imshow(I, cmap="gray"); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)
# for i in anns:
#     [x,y,w,h] = i['bbox']
#     cv2.rectangle(I, (int(x), int(y)), (int(x+w), int(y+h)), 1., 1);
#     #cv2.imshow(I)
# plt.imshow(I, cmap="gray"); plt.axis('off')

In [ ]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print('Running demo for *%s* results.'%(annType))

In [8]:
code_dir = "/data/vision/polina/users/clintonw/code"
mmdet_dir = f"{code_dir}/mmdet"

In [ ]:
job_name = "yolov3_d53_fp16_mstrain-608_273e_coco"
model_type = "yolo"
cfg = f"configs/{model_type}/{job_name}.py"
work_dir = f"/data/vision/polina/scratch/{job_name}"
print(f"sh slurm_train.sh gpu {job_name} {cfg} {work_dir}")

In [ ]:
model_type = "yolo"
job_name = "yolov3_d53_fp16_mstrain-608_273e_coco"
chckpoint = f"{code_dir}/mmdet/checkpoints/yolov3_d53_fp16_mstrain-608_273e_coco_20210517_213542-4bc34944.pth"

cfg = f"configs/{model_type}/{job_name}.py"
res_path = f"results/{job_name}.pkl"
print(f"sh slurm_test.sh gpu {job_name} {cfg} {chckpoint} --out {res_path}")

In [27]:
model_type = "faster_rcnn"
job_name = "faster_rcnn_r50_fpn_mstrain_3x_coco"
chckpoint = f"{code_dir}/mmdet/checkpoints/faster_rcnn_r50_fpn_mstrain_3x_coco_20210524_110822-e10bd31c.pth"

cfg = f"configs/{model_type}/{job_name}.py"
res_path = f"results/{job_name}.pkl"
print(f"./tools/slurm_test.sh gpu {job_name} {cfg} {chckpoint} --out {res_path}")

./tools/slurm_test.sh gpu faster_rcnn_r50_fpn_mstrain_3x_coco configs/faster_rcnn/faster_rcnn_r50_fpn_mstrain_3x_coco.py /data/vision/polina/users/clintonw/code/mmdet/checkpoints/faster_rcnn_r50_fpn_mstrain_3x_coco_20210524_110822-e10bd31c.pth --out results/faster_rcnn_r50_fpn_mstrain_3x_coco.pkl


In [9]:
import dill as pickle
import os

In [16]:
with open(os.path.join(mmdet_dir, res_path), "rb") as f:
    out = pickle.load(f)

In [23]:
len(out)

5000

In [22]:
len(out[0])

80

In [30]:
batch1 = out[0]

In [ ]:
GPUS=16 ./tools/slurm_test.sh gpu mask_r50_1x_test configs/mask_rcnn_r50_fpn_1x_coco.py /nfs/xxxx/mask_rcnn_r50_fpn_1x

### sprelnet

In [ ]:
from sprelnet import train

from sprelnet import util, losses, wab, visualize
from sprelnet.data import mnist, pixels
from sprelnet.networks.relations import get_relnet
from sprelnet.networks.unet import get_unet
from sprelnet.networks.adversarial import get_adv_sprelnet
from sprelnet.networks.contrastive import get_contra_net
from sprelnet.networks.patch_net import get_patch_net

In [ ]:
class cmd_args:
    job_id = "van_pix"
    config_path = f"/data/vision/polina/users/clintonw/code/sprelnet/configs/{job_id}.yaml"
    p = f"{util.base_job_dir}/{job_id}"
    if not os.path.exists(p):
        os.makedirs(p)
with open(cmd_args.config_path, 'r') as stream:
    args = yaml.safe_load(stream)

run, args = train.prep_run(cmd_args, args)

if args["data loading"]["dataset"] == "MNIST grid":
    dataset = mnist.get_multi_mnist(run=run)
elif args["data loading"]["dataset"] == "pixels":
    dataset = pixels.get_pixel_dataset(run=run)
else:
    raise NotImplementedError("bad dataset")

dataloaders = util.get_dataloaders(dataset, batch_size=args["data loading"]["batch size"])

In [ ]:
if args["network"]["type"] in ["adversarial", "vanilla"]:
    get_net_fxn = get_adv_sprelnet
    train_fxn = train.train_main_relnet
elif args["network"]["type"] == "unet":
    get_net_fxn = get_unet
    train_fxn = train.train_unet
elif args["network"]["type"] == "relnet only":
    get_net_fxn = get_relnet
    train_fxn = train.train_relnet
elif args["network"]["type"] == "contrastive":
    get_net_fxn = get_contra_net
    train_fxn = train.train_contranet
elif args["network"]["type"] == "iterative with attention over patch proposals":
    get_net_fxn = get_patch_net
    train_fxn = train.train_patchnet
else:
    raise NotImplementedError("bad network type")

network = get_net_fxn(net_HPs=args["network"], dataset=dataset)
paths=args["paths"]
loss_settings=args["loss"]
optimizer_settings=args["optimizer"]

### Explore JSON annotations

In [21]:
annFile = osp.join(mnist_grid_dir, "val/annotation_coco.json")

In [ ]:
cd /data/vision/polina/scratch/clintonw/datasets/mnist_grid
mv val val2
mv train train2

In [ ]:
instance["segmentation"]

In [ ]:
with open(annFile, "r") as f:
    json_object = json.load(f)

for i, instance in enumerate(json_object["annotations"]):
    if len(instance["segmentation"][0]) == 4:
        print("instance number", i, "raises arror:", instance["segmentation"])

In [ ]:
annFile = osp.join(mnist_grid_dir, "val/annotation_coco.json")
coco = COCO(annFile)

In [3]:
coco_ids = coco.getImgIds()
coco_id = np.random.choice(coco_ids)
img_info = coco.loadImgs([int(coco_id)])[0]
img = io.imread('%s/%s/%s'%(mnist_grid_dir,"val",img_info['file_name']), as_gray=True)
annIds = coco.getAnnIds(imgIds=img_info['id'], iscrowd=None)
mnist_anns = coco.loadAnns(annIds)

In [ ]:
mnist_anns[0]["segmentation"]

In [7]:
annFile = osp.join(coco_dir, "annotations/instances_val2017.json")
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.90s)
creating index...
index created!


In [8]:
coco_ids = coco.getImgIds()
coco_id = np.random.choice(coco_ids)
img_info = coco.loadImgs([int(coco_id)])[0]
img = io.imread('%s/%s/%s'%(coco_dir,"val2017",img_info['file_name']), as_gray=True)
annIds = coco.getAnnIds(imgIds=img_info['id'], iscrowd=None)
coco_anns = coco.loadAnns(annIds)

In [ ]:
coco_anns[0]["segmentation"]

In [ ]:
model_artifact = wandb.Artifact("sprelnet", type="model", description="segmenter with spatial relations")
model_artifact.add_dir(paths["model weights directory"])

wab.define_metrics(["test loss", "test dice"])

G = network.G
relnet = network.relnet
if network.type == "adversarial":
    D = network.D_XY
loss_weights = loss_settings["weights"]

# setup dataloaders
batch_size = dataloaders["train"].batch_size
N_train = len(dataset["train datapoints"])
N_test = len(dataset["test datapoints"])
bce_loss = losses.get_bce_loss(dataset)

G_optim = bnb.optim.Adam8bit(G.parameters(), lr=float(optimizer_settings["G learning rate"]))
if network.type == "adversarial":
    DR_optim = bnb.optim.Adam8bit(itertools.chain(relnet.parameters(), D.parameters()),
        lr=float(optimizer_settings["D learning rate"]))
else:
    DR_optim = bnb.optim.Adam8bit(relnet.parameters(), lr=float(optimizer_settings["relnet learning rate"]))


for epoch in range(1, optimizer_settings["epochs"]+1):
    pxy_true_sum, pxy_fake_sum, rel_true_sum, rel_fake_sum, seg_loss_sum = 0,0, 0,0, 0
    n_batches = math.ceil(N_train/batch_size)
    rel_weight = losses.determine_loss_weight("relation score", epoch=epoch, loss_settings=loss_settings)

    for batch in dataloaders["train"]:
        X,Y_gt = batch
        Y_logits = G(X)
        seg_loss = bce_loss(Y_logits, Y_gt)

        Y_hat = torch.sigmoid(Y_logits)
        if network.type == "adversarial":
            p_xy_fake = D(X, Y_hat).mean()
            pxy_fake_sum += p_xy_fake.item()

        rel_fake = relnet(Y_hat).mean()
        rel_fake_sum += rel_fake.item()
        seg_loss_sum += seg_loss.item()

        if network.type == "adversarial":
            G_loss = seg_loss * loss_weights["cross entropy"] + rel_fake * rel_weight - p_xy_fake
        else:
            G_loss = seg_loss * loss_weights["cross entropy"] + rel_fake * rel_weight

        G_optim.zero_grad()
        G_loss.backward(retain_graph=True)
        G_optim.step()




        if network.type == "adversarial":
            p_xy_true = D(X, Y_gt).mean()
            p_xy_fake = D(X, Y_hat.detach()).mean()
            pxy_true_sum += p_xy_true.item()
        rel_true = relnet(Y_gt).mean() * loss_weights["relation score"]
        rel_true_sum += rel_true.item()

        if network.type == "adversarial":
            DR_loss = p_xy_fake - p_xy_true + rel_true
        else:
            DR_loss = rel_true

        DR_optim.zero_grad()
        DR_loss.backward()
        sparse_reg, smooth_reg = losses.get_multiscale_kernel_regs(relnet, loss_weights)
        sparse_reg.backward()
        smooth_reg.backward()
        util.mask_identity_grad_in_kernel(relnet)
        DR_optim.step()
        break
    break

## Slurm

### classic sprelnet

In [ ]:
sh script_train.sh adv_mni
sh script_train.sh patch_mni
sh script_train.sh unet_mni
sh script_train.sh van_pix
sh script_train.sh van_mni
sh script_train.sh rel_dbug

In [ ]:
util.end_slurm_jobs(["van_mni"])

In [ ]:
#util.rename_job("rel_mni", "rel_mni_01")
util.delete_job_outputs("van_pix")

In [ ]:
util.print_job_log("unet_mni")

In [ ]:
wab.clear_failed_runs()

### MMLab

In [ ]:
## COCO download
https://gist.github.com/mkocabas/a6177fc00315403d31572e17700d7fd9
cd images
wget http://images.cocodataset.org/zips/train2017.zip
wget http://images.cocodataset.org/zips/val2017.zip
wget http://images.cocodataset.org/zips/test2017.zip
wget http://images.cocodataset.org/zips/unlabeled2017.zip
unzip -n val2017.zip
cd ../
wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
wget http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip
wget http://images.cocodataset.org/annotations/image_info_test2017.zip
wget http://images.cocodataset.org/annotations/image_info_unlabeled2017.zip
unzip

In [ ]:
sh tools/clintonw_train.sh faster_rcnn faster_rcnn_r50_fpn_1x_coco
sh tools/clintonw_train.sh faster_rcnn sprel_frcnn_r50_fpn_1x_coco
sh tools/clintonw_train.sh mnist_grid faster_rcnn_r50_fpn_1x_mnist
sh tools/clintonw_train.sh mnist_grid sprel_frcnn_r50_fpn_1x_mnist

sh tools/clintonw_train.sh mask_rcnn sprel_mrcnn_r50_fpn_1x_coco
sh tools/clintonw_train.sh mnist_grid mask_rcnn_r50_fpn_1x_mnist
sh tools/clintonw_train.sh mnist_grid sprel_mrcnn_r50_fpn_1x_mnist

sh tools/clintonw_train.sh cascade_rcnn cascade_mask_rcnn_r50_fpn_1x_coco
sh tools/clintonw_train.sh mnist_grid cascade_mask_rcnn_r50_fpn_1x_mnist
sh tools/clintonw_train.sh mnist_grid srl_cascade_mask_rcnn_r50_fpn_1x_mnist

sh tools/clintonw_train.sh mnist_grid yolov3_d53_fp16_mstrain-608_273e_mnist
sh tools/clintonw_train.sh mnist_grid sprel_yolov3_d53_fp16_mstrain-608_273e_mnist
sh tools/clintonw_train.sh yolo yolov3_d53_fp16_mstrain-608_273e_coco
sh tools/clintonw_train.sh yolo sprel_yolov3_d53_fp16_mstrain-608_273e_coco



cp results/srl_cascade_mask_rcnn_r50_fpn_1x_mnist_train/latest.pth checkpoints/cascade_mask_rcnn_r50_fpn_1x_mnist_01.pth

sh tools/clintonw_test.sh mnist_grid cascade_mask_rcnn_r50_fpn_1x_mnist 01
sh tools/clintonw_test.sh faster_rcnn faster_rcnn_r50_fpn_1x_coco 20200130-047c8118

sh tools/slurm_analyze.sh mnist_grid sprel_mrcnn_r50_fpn_1x_mnist 01

sh tools/bbox_results.sh mnist_grid mask_rcnn_r50_fpn_1x_mnist 01 \
    /data/vision/polina/scratch/clintonw/datasets/mnist_grid/val/annotation_coco.json
sh tools/bbox_results.sh faster_rcnn sprel_frcnn_r50_fpn_1x_coco 01 \
    /data/vision/polina/scratch/clintonw/datasets/coco/annotations/instances_val2017.json

sh tools/segm_results.sh mnist_grid sprel_mrcnn_r50_fpn_1x_mnist 01 \
    /data/vision/polina/scratch/clintonw/datasets/mnist_grid/val/annotation_coco.json

python tools/analysis_tools/get_flops.py ${CONFIG_FILE} --shape ${INPUT_SHAPE}

In [ ]:
python tools/train.py configs/mnist_grid/srl_cascade_mask_rcnn_r50_fpn_1x_mnist.py \
        --work-dir=results/manual_train --deterministic

In [ ]:
importlib.reload(omm_util)

#### Job analysis

In [ ]:
job = "srl_cascade_mask_rcnn_r50_fpn_1x_mnist_train"
logs, errs = omm_util.print_job_log(job), omm_util.print_job_err(job)
logs, errs
errs[-25:]

In [ ]:
job_name = "sprel_mrcnn_r50_fpn_1x_mnist_01_test"
# path = f"{base_job_dir}/{job_name}/imgs"
# if not osp.exists(path):
#     raise ValueError(f"{path} does not exist")
# img_paths = glob.glob(path+"/*")
img = omm_util.show_sample_result(job_name)

In [ ]:
metrics = ["bbox_mAP", "bbox_mAP_50", "bbox_mAP_75"]
jobs = ["faster_rcnn_r50_fpn_1x_mnist_01", "sprel_frcnn_r50_fpn_1x_mnist_01",
    "mask_rcnn_r50_fpn_1x_mnist_01", "sprel_mrcnn_r50_fpn_1x_mnist_01"]
table = omm_util.collate_metrics_for_jobs(metrics, jobs)
table

In [ ]:
compare_jobs("faster_rcnn_r50_fpn_1x_mnist_01_test", "sprel_frcnn_r50_fpn_1x_mnist_01_test")

In [ ]:
## plot metric for training job
train_job = "frc_mn2"
logpath = glob.glob(f"{base_job_dir}/{train_job}/*.log.json")[0]
metrics = "bbox_mAP" #loss_cls"
outpath = osp.join(osp.dirname(logpath), f"{metrics}.pdf")
print(f"""python tools/analysis_tools/analyze_logs.py plot_curve \
{logpath} --keys {metrics} --legend {metrics} --out {outpath}""")

In [ ]:
annFile = osp.join(mnist_grid_dir, "val/annotation_coco.json")
mnist_coco = COCO(annFile)
def truncate_(s):
    return s[:s.find('_')]

In [ ]:
job = "faster_rcnn_r50_fpn_1x_mnist_01_test"
img_paths = glob.glob(mmdet_job_dir+f"/{job}/imgs/bad/*")
img_path = img_paths[0]
test_img = plt.imread(img_path)
coco_id = truncate_(osp.basename(img_path))
raw_img, anns = mnist.fetch_mnist_grid_img(coco_id, with_anns=True, coco_object=mnist_coco)
plt.imshow(raw_img, cmap="gray"); plt.axis('off')
coco.showAnns(anns)

In [ ]:
job = "faster_rcnn_r50_fpn_1x_mnist_01_test"
img_paths = glob.glob(mmdet_job_dir+f"/{job}/imgs/good/*")
img_path = img_paths[0]
test_img = plt.imread(img_path)
coco_id = truncate_(osp.basename(img_path))
raw_img, anns = mnist.fetch_mnist_grid_img(coco_id, with_anns=True, coco_object=mnist_coco)
plt.imshow(raw_img, cmap="gray"); plt.axis('off')
coco.showAnns(anns)

In [ ]:
job = "mask_rcnn_r50_fpn_1x_mnist_01_test"
glob.glob(mmdet_job_dir+f"/{job}/*")

In [ ]:
job = "mask_rcnn_r50_fpn_1x_mnist_01_test"
with open(f"{mmdet_job_dir}/{job}/logs.out", "r") as f:
    out = f.readlines()
out[-10:]

In [ ]:
job = "sprel_mrcnn_r50_fpn_1x_mnist_01_test"
with open(f"{mmdet_job_dir}/{job}/logs.out", "r") as f:
    out = f.readlines()
out[-10:]

In [ ]:
# https://cocodataset.org/#detection-eval
# C75: PR at IoU=.75 (AP at strict IoU), area under curve corresponds to APIoU=.75 metric.
# C50: PR at IoU=.50 (AP at PASCAL IoU), area under curve corresponds to APIoU=.50 metric.
# Loc: PR at IoU=.10 (localization errors ignored, but not duplicate detections). All remaining settings use IoU=.1.
# Sim: PR after supercategory false positives (fps) are removed. Specifically, any matches to objects with a different class label but that belong to the same supercategory don't count as either a fp (or tp). Sim is computed by setting all objects in the same supercategory to have the same class label as the class in question and setting their ignore flag to 1. Note that person is a singleton supercategory so its Sim result is identical to Loc.
# Oth: PR after all class confusions are removed. Similar to Sim, except now if a detection matches any other object it is no longer a fp (or tp). Oth is computed by setting all other objects to have the same class label as the class in question and setting their ignore flag to 1.
# BG: PR after all background (and class confusion) fps are removed. For a single category, BG is a step function that is 1 until max recall is reached then drops to 0 (the curve is smoother after averaging across categories).
# FN: PR after all remaining errors are removed (trivially AP=1).
# The area under each curve is shown in brackets in the legend. In the case of the ResNet detector, overall AP at IoU=.75 is .399 and perfect localization would increase AP to .682. Interesting, removing all class confusions (both within supercategory and across supercategories) would only raise AP slightly to .713. Removing background fp would bump performance to .870 AP and the rest of the errors are missing detections (although presumably if more detections were added this would also add lots of fps). In summary, ResNet's errors are dominated by imperfect localization and background confusions.

In [ ]:
job = "mask_rcnn_r50_fpn_1x_mnist_01_test"
classname = "allclass"
img_path = osp.join(mmdet_job_dir, f"{job}/imgs/segm/segm-{classname}-allarea.png")
plot = plt.imread(img_path)
plt.imshow(plot, cmap="gray"); plt.axis('off')

In [ ]:
job = "sprel_mrcnn_r50_fpn_1x_mnist_01_test"
classname = "allclass"
img_path = osp.join(base_job_dir, f"{job}/imgs/segm/segm-{classname}-allarea.png")
plot = plt.imread(img_path)
plt.imshow(plot, cmap="gray"); plt.axis('off')

In [ ]:
job = "faster_rcnn_r50_fpn_1x_mnist_01_test"
classname = "allclass"
img_path = osp.join(mmdet_job_dir, f"{job}/imgs/bbox/bbox-{classname}-allarea.png")
plot = plt.imread(img_path)
plt.imshow(plot, cmap="gray"); plt.axis('off')

In [ ]:
job = "sprel_frcnn_r50_fpn_1x_mnist_01_test"
classname = "allclass"
img_path = osp.join(base_job_dir, f"{job}/imgs/bbox/bbox-{classname}-allarea.png")
plot = plt.imread(img_path)
plt.imshow(plot, cmap="gray"); plt.axis('off')

In [ ]:
path = base_proj_dir+"/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_mnist.py"